In [104]:
'''
This contains code for generating the table "Comparison to state-of-the art counterfactual example generation techniques in terms of explanation time t, sparsity s, L1-Norm 𝛿1, L2-Norm 𝛿2, and validity %. (*) denotes datasets where RFOCSE necessitated uncapped explanation time."

A CSV and TEX version will be generated, adjustments to the LaTeX table fontsize and table width may be neccessary

Experiment results files needed: CompareMethods

Results used in the paper are provided in "../results/final" if generating new results run each experiment and update the results paths below
'''
# path to each result file
results_path = "../results/compare_methods_gb.csv"

# path to output the figure
export_figures = False
output_dir = "./compare-methods/"
table_save_name = "compare_methods"

In [105]:
import os
import re
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

sys.path.append(os.path.abspath("../"))
#from dataset import DS_NAMES
#from utilities.figure_tools import (get_latest_results_directory, load_results,
                                    #make_fig_directory)

cmap = plt.get_cmap("Set1")
colors = cmap.colors
if export_figures and not os.path.isdir(output_dir):
    os.makedirs(output_dir)

In [106]:
all_results = pd.read_csv(results_path)
all_results = all_results[all_results["n_trees"] == 10]
all_results = all_results.groupby(["dataset", "explainer"]).mean().reset_index()

found_expl = list(all_results["explainer"].unique())
found_ds = list(all_results["dataset"].unique())
print("explainers:", found_expl)
print("datasets", found_ds)
all_results.head()
ds_order = ["compas", "glass", "vertebral"]
expl_order = ["FACETIndex", "MACE", "OCEAN", "RFOCSE", "AFT"]

explainers: []
datasets []


In [107]:
# # facet_dists.head()
# norm_data = all_results.copy()
# for expl in norm_data["explainer"].unique():
#     for ds in norm_data["dataset"].unique():
#         facet_idx = (all_results["explainer"] == "FACETIndex") & (all_results["dataset"] == ds)
# #         facet_dist = all_results[facet_idx]["avg_dist"]

# #         match_idx = (all_results["explainer"] == "FACETIndex") & (all_results["dataset"] == ds)
# #         norm_data[match_idx] = norm_data[match_idx] / facet_dist

# # norm_data["avg_dist"] = norm_data["avg_dist"] / facet_dists
# # norm_data.head()

In [108]:
opt_vals = {}
bold_opt = False
domin = {
    "sample_time": True,
    "avg_length": True,
    "avg_dist": True,
    "avg_manhattan": True,
    "per_valid": False,
}
metrics = ["sample_time", "avg_length", "avg_dist", "avg_manhattan", "per_valid"]


for ds in all_results["dataset"].unique():
    idx = (all_results["dataset"] == ds)
    opt_vals[ds] = {}
    for m in metrics:
        if domin[m]:
            opt_vals[ds][m] = all_results[idx][m].min()
        else:
            opt_vals[ds][m] = all_results[idx][m].max()

metric_latex = {
    "sample_time": "$t\downarrow$",
    "avg_length": "$\delta_0\downarrow$",
    "avg_manhattan": "$\delta_1\downarrow$",
    "avg_dist": "$\delta_2\downarrow$",
    "per_valid": "$\%\\uparrow$",
}
pretty_names = {
    "FACETIndex": "FACET",
    "MACE": "MACE",
    "AFT": "AFT",
    "OCEAN": "OCEAN",
    "RFOCSE": "RFOCSE",
}
all_metrics = ["sample_time", "avg_length", "avg_manhattan", "avg_dist", "per_valid"]

In [109]:
def df_to_csv_latex(df_source, expls, metrics, fname, include_row_label=True):
    #df_ds = df_source["dataset"].unique()
    df_ds = ["compas", "glass", "vertebral"]
    df = df_source.copy()
    df = df.set_index(["dataset", "explainer"])
    with open(output_dir + fname + ".csv", "w") as csv:
        with open(output_dir + fname + ".tex", "w") as tab:
            # csv header row start
            if include_row_label:
                csv.write("dataset,")
            # tab header row start
            tab.write("\\begin{table*}[t]\n\small\n\centering\n\\begin{tabularx}{0.95\\textwidth}{")
            if include_row_label:
                tab.write("|X")
            tab.write("|")
            for expl in expls:
                for m in metrics:
                    tab.write("c")
                tab.write("|")
            tab.write("}\n\hline")
            if include_row_label:
                tab.write("\\textbf{Dataset}")
            # csv and tab header row
            for expl in expls:
                tab.write(" & \multicolumn{5}{c|}{\\textbf{" + pretty_names[expl] +"}}")
                for m in metrics:
                    csv.write(pretty_names[expl] + ",")
                csv.write(",")
            csv.write("\n")
            tab.write(" " + re.escape("\\") + "\n")
            # header row two
            csv.write(",")
            for expl in expls:
                for m in metrics:
                    csv.write(m + ",")
                    tab.write("& " + metric_latex[m] + " ")
                csv.write(",")
            csv.write("\n")
            tab.write(re.escape("\\") + "\n\hline\n")
            # csv and tab body row
            for ds in df_ds:
                if include_row_label:
                    csv.write(ds + ",")
                    tab.write(ds)
                for expl in expls:
                    for m in metrics:
                        if m == "per_valid":
                            val = df.loc[ds, expl][m] * 100
                            val_str = "{:0.1f}".format(val)
                            csv.write(val_str + ",")
                            if bold_opt and df.loc[ds, expl][m] == opt_vals[ds][m]:
                                tab.write(" & \\textbf{" + val_str + "}")
                            else:
                                tab.write(" & " + val_str)
                        elif m == "avg_length":
                            val = df.loc[ds, expl][m]
                            val_str = "{:0.2f}".format(val)
                            csv.write(val_str + ",")
                            if bold_opt and df.loc[ds, expl][m] == opt_vals[ds][m]:
                                tab.write(" & \\textbf{" + val_str + "}")
                            else:
                                tab.write(" & " + val_str)
                        elif m == "avg_dist":
                            val = df.loc[ds, expl][m] / df.loc[ds, "FACETIndex"][m]
                            val_str = "{:0.2f}".format(val)
                            csv.write(val_str + ",")
                            if bold_opt and df.loc[ds, expl][m] == opt_vals[ds][m]:
                                tab.write(" & \\textbf{" + val_str + "}")
                            else:
                                tab.write(" & " + val_str)
                        elif m == "avg_manhattan":
                            val = df.loc[ds, expl][m] / df.loc[ds, "FACETIndex"][m]
                            val_str = "{:0.2f}".format(val)
                            csv.write(val_str + ",")
                            if bold_opt and df.loc[ds, expl][m] == opt_vals[ds][m]:
                                tab.write(" & \\textbf{" + val_str + "}")
                            else:
                                tab.write(" & " + val_str)
                        elif m == "sample_time":
                            val = df.loc[ds, expl][m]
                            val_str = "{:0.4f}".format(val)
                            csv.write(val_str + ",")
                            if bold_opt and df.loc[ds, expl][m] == opt_vals[ds][m]:
                                tab.write(" & \\textbf{" + val_str + "}")
                            else:
                                tab.write(" & " + val_str)
                        else:
                            val_str = str(df.loc[ds, expl][m])
                            csv.write(val_str + ",")
                            tab.write(" & " + val_str)
                    csv.write(",")
                csv.write("\n")
                tab.write(" " + re.escape("\\") + "\n")
            # tab latex close
            tab.write("\hline\n\end{tabularx}\n")
            tab.write("\caption{Comparison to state-of-the art counterfactual example generation techniques in terms of explanation time $t$, explanation distance $\delta$, and percent of instances successfully explained. ($\\ast$) denotes cases which necessitated uncapped explanation time.}\n")
            tab.write("\label{tab.compare_methods}\n\\vspace{-7mm}\n\end{table*}\n")

In [110]:
def df_to_csv_latex_transponse(df_source, expls, ds_names, metrics, fname, include_row_label=True):
    valid_expls = []
    for e in expls: 
        if e in found_expl:
            valid_expls.append(e)
    valid_expls
    print(expls)

    valid_ds = []
    for ds in ds_names: 
        if ds in found_ds:
            valid_ds.append(ds)
    ds_names = valid_ds
    print(ds_names)

    df_ds = df_source["dataset"].unique()
    df = df_source.copy()
    df = df.set_index(["dataset", "explainer"])

    with open(output_dir + fname + "_transponse" + ".csv", "w") as csv:
        with open(output_dir + fname + "_transponse" + ".tex", "w") as tab:
            # csv header row start
            if include_row_label:
                csv.write("dataset,")
            # tab header row start
            tab.write("\\begin{table*}[t]\n\small\n\centering\n\\begin{tabularx}{0.95\\textwidth}{")
            if include_row_label:
                tab.write("|X")
            tab.write("|")
            for ds in valid_ds:
                for m in metrics:
                    tab.write("c")
                tab.write("|")
            tab.write("}\n\hline")
            if include_row_label:
                tab.write("\\textbf{Dataset}")
            # csv and tab header row
            for ds in valid_ds:
                tab.write(" & \multicolumn{5}{c|}{\\textbf{" + ds.upper() +"}}")
                for m in metrics:
                    csv.write(ds.upper() + ",")
                csv.write(",")
            csv.write("\n")
            tab.write(" " + re.escape("\\") + "\n")
            # header row two
            csv.write(",")
            for ds in valid_ds:
                for m in metrics:
                    csv.write(m + ",")
                    tab.write("& " + metric_latex[m] + " ")
                csv.write(",")
            csv.write("\n")
            tab.write(re.escape("\\") + "\n\hline\n")
            # csv and tab body row
            for expl in valid_expls:
                if include_row_label:
                    csv.write(pretty_names[expl] + ",")
                    tab.write(pretty_names[expl])
                for ds in valid_ds:
                    for m in metrics:
                        if m == "per_valid":
                            val = df.loc[ds, expl][m] * 100
                            val_str = "{:0.1f}".format(val)
                            csv.write(val_str + ",")
                            if bold_opt and df.loc[ds, expl][m] == opt_vals[ds][m]:
                                tab.write(" & \\textbf{" + val_str + "}")
                            else:
                                tab.write(" & " + val_str)
                        elif m == "avg_length":
                            val = df.loc[ds, expl][m]
                            val_str = "{:0.2f}".format(val)
                            csv.write(val_str + ",")
                            if bold_opt and df.loc[ds, expl][m] == opt_vals[ds][m]:
                                tab.write(" & \\textbf{" + val_str + "}")
                            else:
                                tab.write(" & " + val_str)
                        elif m == "avg_dist":
                            val = df.loc[ds, expl][m] / df.loc[ds, "FACETIndex"][m]
                            val_str = "{:0.2f}".format(val)
                            csv.write(val_str + ",")
                            if bold_opt and df.loc[ds, expl][m] == opt_vals[ds][m]:
                                tab.write(" & \\textbf{" + val_str + "}")
                            else:
                                tab.write(" & " + val_str)
                        elif m == "avg_manhattan":
                            val = df.loc[ds, expl][m] / df.loc[ds, "FACETIndex"][m]
                            val_str = "{:0.2f}".format(val)
                            csv.write(val_str + ",")
                            if bold_opt and df.loc[ds, expl][m] == opt_vals[ds][m]:
                                tab.write(" & \\textbf{" + val_str + "}")
                            else:
                                tab.write(" & " + val_str)
                        elif m == "sample_time":
                            val = df.loc[ds, expl][m]
                            val_str = "{:0.4f}".format(val)
                            csv.write(val_str + ",")
                            if bold_opt and df.loc[ds, expl][m] == opt_vals[ds][m]:
                                tab.write(" & \\textbf{" + val_str + "}")
                            else:
                                tab.write(" & " + val_str)
                        else:
                            val_str = str(df.loc[ds, expl][m])
                            csv.write(val_str + ",")
                            tab.write(" & " + val_str)
                    csv.write(",")
                csv.write("\n")
                tab.write(" " + re.escape("\\") + "\n")
            # tab latex close
            tab.write("\hline\n\end{tabularx}\n")
            tab.write("\caption{Comparison to state-of-the art counterfactual example generation techniques in terms of explanation time $t$, explanation distance $\delta$, and percent of instances successfully explained. ($\\ast$) denotes cases which necessitated uncapped explanation time.}\n")
            tab.write("\label{tab.compare_methods}\n\\vspace{-7mm}\n\end{table*}\n")

In [111]:
# df_to_csv_latex_transponse(all_results, ["FACETIndex", "MACE", "OCEAN", "RFOCSE", "AFT"], ["cancer", "glass", "magic", "spambase", "vertebral"], all_metrics, table_save_name, True)

In [112]:
df_to_csv_latex_transponse(all_results, expl_order, ds_order, all_metrics, table_save_name, True)

['FACETIndex', 'MACE', 'OCEAN', 'RFOCSE', 'AFT']
[]


FileNotFoundError: [Errno 2] No such file or directory: './compare-methods/compare_methods_transponse.csv'

In [ ]:
all_results[["dataset", "explainer", "avg_dist", "per_valid", "prep_time", "sample_time"]].pivot(index=["dataset"], columns=["explainer"], values=["per_valid"])

per_valid                               
explainer       AFT FACETIndex   MACE OCEAN RFOCSE
dataset                                           
adult         0.960        1.0  0.685   1.0  0.905
cancer        0.370        1.0  1.000   1.0  1.000
compas        1.000        1.0  1.000   1.0  1.000
credit        0.855        1.0  0.380   1.0  0.350
glass         1.000        1.0  1.000   1.0  1.000
magic         0.990        1.0  1.000   1.0  0.950
spambase      0.760        1.0  1.000   1.0  1.000
vertebral     1.000        1.0  1.000   1.0  1.000

In [ ]:
all_results[["dataset", "explainer", "avg_dist", "per_valid", "prep_time", "sample_time"]].pivot(index=["dataset"], columns=["explainer"], values=["avg_dist"])

avg_dist                                          
explainer       AFT FACETIndex       MACE     OCEAN    RFOCSE
dataset                                                      
adult      0.139197   0.173682  14.131749  0.515992  0.099988
cancer     0.194152   0.153822   1.365519  0.156526  0.164461
compas     0.109951   0.140730   0.868542  0.582896  0.140605
credit     0.345228   0.282330   1.628337  0.557298  0.060658
glass      0.085238   0.085770   0.749309  0.081336  0.087032
magic      0.118433   0.095715   0.728178  0.100905  0.099288
spambase   0.024882   0.008207   3.042850  0.005249  0.037012
vertebral  0.080469   0.066099   0.458792  0.072163  0.055250

In [ ]:
all_results[["dataset", "explainer", "avg_dist", "per_valid", "prep_time", "sample_time"]].pivot(index=["dataset"], columns=["explainer"], values=["sample_time"])

sample_time                                              
explainer         AFT FACETIndex        MACE     OCEAN       RFOCSE
dataset                                                            
adult        0.002618   0.107182  107.405868  0.585761    56.582770
cancer       0.001936   0.094203    2.221864  0.680870  1936.015359
compas       0.003131   0.002038    2.175595  0.603602     1.864458
credit       0.002332   0.128122  190.211784  1.720524   202.537994
glass        0.001786   0.005670  252.511365  1.113207     3.776954
magic        0.003586   0.002980    2.472205  1.790337   147.044462
spambase     0.002658   0.200871   19.912989  0.463495  2208.397540
vertebral    0.002121   0.002555   10.676982  0.916410     2.475996